In [2]:
import spacy
import pandas as pd
from spacy.tokens import DocBin

import re # remove links

import functions


c:\Users\joseph\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model 0 preprocessing - geolocation (colombia)

In [5]:
#Exporting train ant test data for Colombia (will be used for training)
input_texts_df = pd.read_csv("./../data/texts.csv")

In [6]:
input_texts_df = input_texts_df[input_texts_df['text']!='error reading status']

In [8]:
input_texts_df['text'] = input_texts_df['text'].apply(functions.tweet_text_clean)
input_texts_df['text']  = input_texts_df['text'].str.strip()

In [17]:
print("With duplicates:",len(input_texts_df))
print("Removing duplicates:",len(input_texts_df['text'].unique()))

With duplicates 4144218
Removing duplicates: 787204


In [18]:
input_texts_df.drop_duplicates(subset = 'text', inplace = True)

In [19]:
countries = ['Colombia', 'no_country']
def group_by_country(country):
    if country not in countries:
        country = 'other'
    return country

In [20]:
input_texts_df['country_train'] = input_texts_df['country'].str.strip().fillna('no_country').apply(group_by_country)

In [21]:
input_texts_df['country_train'].value_counts()

other         355112
no_country    296545
Colombia      135547
Name: country_train, dtype: int64

In [22]:
input_texts_df = input_texts_df[input_texts_df['country_train']!= 'no_country']

In [23]:
input_texts_df['text'] = input_texts_df['text'].astype(str)

C:\Users\joseph\AppData\Local\Temp\ipykernel_10132\690597189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_texts_df['text'] = input_texts_df['text'].astype(str)


In [24]:
input_data = functions.eq_division(input_texts_df, n_sample = 135500, cats_col_name = 'country_train')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)


In [25]:
train_col, test_col = functions.random_select(input_data = input_data, cats_column_name = 'country_train')
train_col.to_excel('./model0_geolocation/train.xlsx', index = False), test_col.to_excel('./model0_geolocation/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [26]:
train_col['country_train'].value_counts()

other       94850
Colombia    94850
Name: country_train, dtype: int64

In [27]:
print(len(train_col))
print(len(test_col))

189700
81300


In [28]:
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, cat_col_name= 'country_train', text_col_name='text') #using the spacy doc arquitecture for our data

In [29]:
DocBin(docs = train_docs_col).to_disk("./model0_geolocation/train.spacy") 

# Model 1 preprocessing - valid tweets

In [58]:
#Exporting train ant test data for Colombia (will be used for training)
input_data_col = pd.read_excel("./../data/4k_tweets/manually_labeled_tweets_colombia.xlsx")
#input_data_col['paragraph'] = input_data_col['paragraph'].astype(str)
#input_data_col['works'] = input_data_col['works'].astype(str)


In [59]:
input_data_col['text_es_clean'] = input_data_col['text_es'].apply(functions.tweet_text_clean)
input_data_col.drop_duplicates(subset = 'text_es_clean', inplace = True)

In [60]:
input_data_col.to_csv('./../data/4k_tweets/cleaned_tweets_colombia.csv', index = False)

In [61]:
input_data_col['valid'] = input_data_col['valid'].astype(str)
input_data_col[input_data_col['valid']=="1"].to_csv('./../nlp_models/model1_valid/valid_cleaned_tweets_colombia.csv', index = False)
input_data_col['valid'].value_counts()


1    2986
0    1568
Name: valid, dtype: int64

In [62]:
input_data_col = functions.eq_division(input_data_col, n_sample = 1550, cats_col_name = 'valid')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)


In [63]:
train_col, test_col = functions.random_select(input_data = input_data_col, cats_column_name = 'valid')
train_col.to_excel('./model1_valid/train.xlsx', index = False), test_col.to_excel('./model1_valid/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [64]:
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, cat_col_name= 'valid', text_col_name='text_es_clean') #using the spacy doc arquitecture for our data

# Model 2 preprocessing - blaming

In [71]:
input_data_col = pd.read_csv('./../nlp_models/model1_valid/valid_cleaned_tweets_colombia.csv')
print(input_data_col['referred_to'].value_counts())

migrants       1526
government      596
geopolitics     329
migration       296
locals           95
Media            38
Other            11
Name: referred_to, dtype: int64


In [73]:
final_frusts = ['migrants', 'government', 'geopolitics', 'migration']
uniform_data = functions.eq_division(input_data_col, final_frusts, n_sample = 295, cats_col_name='referred_to')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academi

In [74]:
train_col, test_col = functions.random_select(input_data = uniform_data, cats_column_name = 'referred_to')
train_col.to_excel('./model2_blaming/train.xlsx', index = False), test_col.to_excel('./model2_blaming/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [75]:
#nlp = spacy.load("es_core_news_sm")
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, text_col_name='text_es_clean', cat_col_name= 'referred_to') #using the spacy doc arquitecture for our data

In [76]:
DocBin(docs = train_docs_col).to_disk("./model2_blaming/train.spacy") 

# Model 3.1 preprocessing - tone

In [3]:
input_data = pd.read_csv('./../data/4k_tweets/cleaned_tweets_colombia.csv')

In [5]:
input_data.dropna(subset = ['tone'], inplace = True)

In [8]:
text_tones = []
for tone in input_data['tone']:
    if tone <0:
        text_tones.append('negative')
    elif tone ==0:
        text_tones.append('neutral')
    else:
        text_tones.append('positive')
input_data['text_tone'] = text_tones

In [9]:
input_data['text_tone'].value_counts()

negative    1570
neutral      770
positive     646
Name: text_tone, dtype: int64

In [10]:
uniform_data = functions.eq_division(input_data, n_sample = 640, cats_col_name='text_tone')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)


In [12]:
train_col, test_col = functions.random_select(input_data = uniform_data, cats_column_name = 'text_tone')
train_col.to_excel('./model3_1_tone/train.xlsx', index = False), test_col.to_excel('./model3_1_tone/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [13]:
#nlp = spacy.load("es_core_news_sm")
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, text_col_name='text_es_clean', cat_col_name= 'text_tone') #using the spacy doc arquitecture for our data

In [14]:
DocBin(docs = train_docs_col).to_disk("./model3_1_tone/train.spacy") 

# Model 3.2 preprocessing - negativeness

In [15]:
input_data = pd.read_csv('./../data/4k_tweets/cleaned_tweets_colombia.csv')

In [16]:
input_data.dropna(subset = ['tone'], inplace = True)
input_data = input_data[input_data['tone']<0]

In [18]:
input_data['tone'] = input_data['tone'].astype(int).astype(str)
input_data['tone'].value_counts()

-1    819
-2    540
-3    211
Name: tone, dtype: int64

In [19]:
uniform_data = functions.eq_division(input_data, n_sample = 210, cats_col_name='tone')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)


In [20]:
train_col, test_col = functions.random_select(input_data = uniform_data, cats_column_name = 'tone')
train_col.to_excel('./model3_2_negativeness/train.xlsx', index = False), test_col.to_excel('./model3_2_negativeness/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [21]:
#nlp = spacy.load("es_core_news_sm")
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, text_col_name='text_es_clean', cat_col_name= 'tone') #using the spacy doc arquitecture for our data

In [22]:
DocBin(docs = train_docs_col).to_disk("./model3_2_negativeness/train.spacy") 

# Model 3.3 preprocessing - positiveness

In [6]:
input_data = pd.read_csv('./../data/4k_tweets/cleaned_tweets_colombia.csv')

In [10]:
input_data.dropna(subset = ['tone'], inplace = True)
input_data = input_data[input_data['tone']>0]

In [11]:
input_data['pos_tone'] = input_data['tone'].astype(int).astype(str)
input_data['pos_tone'].value_counts()

C:\Users\joseph\AppData\Local\Temp\ipykernel_988\4237494380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['pos_tone'] = input_data['tone'].astype(int).astype(str)


1    445
2    157
3     44
Name: pos_tone, dtype: int64

In [12]:
uniform_data = functions.eq_division(input_data, n_sample = 44, cats_col_name='pos_tone')

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data = divided_data.append(sub_data)


In [13]:
train_col, test_col = functions.random_select(input_data = uniform_data, cats_column_name = 'pos_tone')
train_col.to_excel('./model5_positiveness/train.xlsx', index = False), test_col.to_excel('./model5_positiveness/test.xlsx', index = False)

c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Search (10M)\nlp_models\functions.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train)
c:\Users\joseph\OneDrive - Universidad del Norte\Scripts VMASC\Academic Research Twitter Searc

(None, None)

In [14]:
#nlp = spacy.load("es_core_news_sm")
train_docs_col = functions.make_docs(spacy.load("es_core_news_sm"), train_col, text_col_name='text_es_clean', cat_col_name= 'pos_tone') #using the spacy doc arquitecture for our data

In [15]:
DocBin(docs = train_docs_col).to_disk("./model5_positiveness/train.spacy") 